In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
drive = '/content/drive/MyDrive/DataAnalysisKnock/5章/'

# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [20]:
import pandas as pd

customer = pd.read_csv(drive + 'customer_join.csv')
uselog_months = pd.read_csv(drive + 'use_log_months.csv')

In [21]:
year_months = list(uselog_months['年月'].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['年月']==year_months[i]]
    tmp.rename(columns={'count': 'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['年月']==year_months[i-1]]
    del tmp_before['年月']
    tmp_before.rename(columns={'count': 'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)

uselog.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42：退会前月の退会顧客データを作成しよう

In [22]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer['is_deleted']==1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)
    
exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"])
exit_customer['年月'] = exit_customer['exit_date'].dt.strftime('%Y%m')
uselog['年月'] = uselog['年月'].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id', '年月'], how='left')
print(len(uselog))
exit_uselog.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

33851


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [23]:
exit_uselog = exit_uselog.dropna(subset=['name'])
print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


### ノック43：継続顧客のデータを作成しよう

In [24]:
conti_customer = customer.loc[customer['is_deleted']==0]
conti_uselog = pd.merge(uselog, conti_customer, on=['customer_id'], how='left')
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=['name'])
print(len(conti_uselog))

33851
27422


In [25]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset='customer_id')
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201810,HD744684,4,7.0,XXXX,C03,M,2017-04-01,NaN,CA1,...,ナイト,6000.0,通常,6.250000,6.0,11.0,4.0,1.0,2019-04-30,24.0
1,201812,HI471204,5,4.0,XXX,C01,M,2016-02-01,NaN,CA1,...,オールタイム,10500.0,通常,4.333333,4.0,7.0,1.0,1.0,2019-04-30,38.0
2,201811,PL152568,8,2.0,XXXXX,C03,F,2015-05-01,NaN,CA1,...,ナイト,6000.0,通常,4.666667,5.0,8.0,2.0,1.0,2019-04-30,47.0
3,201901,AS099984,5,5.0,XXX,C01,F,2015-06-01,NaN,CA1,...,オールタイム,10500.0,通常,4.500000,4.5,7.0,2.0,1.0,2019-04-30,46.0
4,201903,TS466629,7,11.0,XXXXX,C03,M,2018-06-12,NaN,CA2,...,ナイト,6000.0,入会費半額,8.500000,8.0,11.0,5.0,1.0,2019-04-30,10.0


In [26]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201810,HD744684,4,7.0,XXXX,C03,M,2017-04-01,NaN,CA1,...,6000.0,通常,6.250000,6.0,11.0,4.0,1.0,2019-04-30,24.0,NaT
1,201812,HI471204,5,4.0,XXX,C01,M,2016-02-01,NaN,CA1,...,10500.0,通常,4.333333,4.0,7.0,1.0,1.0,2019-04-30,38.0,NaT
2,201811,PL152568,8,2.0,XXXXX,C03,F,2015-05-01,NaN,CA1,...,6000.0,通常,4.666667,5.0,8.0,2.0,1.0,2019-04-30,47.0,NaT
3,201901,AS099984,5,5.0,XXX,C01,F,2015-06-01,NaN,CA1,...,10500.0,通常,4.500000,4.5,7.0,2.0,1.0,2019-04-30,46.0,NaT
4,201903,TS466629,7,11.0,XXXXX,C03,M,2018-06-12,NaN,CA2,...,6000.0,入会費半額,8.500000,8.0,11.0,5.0,1.0,2019-04-30,10.0,NaT


### ノック44：予測する月の在籍期間を作成しよう

In [27]:
predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['年月'], format='%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
    predict_data['period'][i] = int(delta.years*12 * delta.months)

predict_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201810,HD744684,4,7.0,XXXX,C03,M,2017-04-01,NaN,CA1,...,6.250000,6.0,11.0,4.0,1.0,2019-04-30,24.0,NaT,72,2018-10-01
1,201812,HI471204,5,4.0,XXX,C01,M,2016-02-01,NaN,CA1,...,4.333333,4.0,7.0,1.0,1.0,2019-04-30,38.0,NaT,240,2018-12-01
2,201811,PL152568,8,2.0,XXXXX,C03,F,2015-05-01,NaN,CA1,...,4.666667,5.0,8.0,2.0,1.0,2019-04-30,47.0,NaT,216,2018-11-01
3,201901,AS099984,5,5.0,XXX,C01,F,2015-06-01,NaN,CA1,...,4.500000,4.5,7.0,2.0,1.0,2019-04-30,46.0,NaT,252,2019-01-01
4,201903,TS466629,7,11.0,XXXXX,C03,M,2018-06-12,NaN,CA2,...,8.500000,8.0,11.0,5.0,1.0,2019-04-30,10.0,NaT,0,2019-03-01


### ノック45：欠損値を除去しよう

In [28]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               266
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [29]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2628
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2628
period                  0
now_date                0
dtype: int64

### ノック46：文字列型の変数を処理できるように整形しよう

In [30]:
target_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flg', 'period', 'is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,ナイト,M,7.0,1.0,72,0.0
1,通常,オールタイム,M,4.0,1.0,240,0.0
2,通常,ナイト,F,2.0,1.0,216,0.0
3,通常,オールタイム,F,5.0,1.0,252,0.0
4,入会費半額,ナイト,M,11.0,1.0,0,0.0


In [31]:
predict_data = pd.get_dummies(predict_data, drop_first=True)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費無料,campaign_name_通常,class_name_デイタイム,class_name_ナイト,gender_M
0,7.0,1.0,72,0.0,0,1,0,1,1
1,4.0,1.0,240,0.0,0,1,0,0,1
2,2.0,1.0,216,0.0,0,1,0,1,0
3,5.0,1.0,252,0.0,0,1,0,0,0
4,11.0,1.0,0,0.0,0,0,0,1,1


### ノック47：決定木を用いて退会予測モデルを作成してみよう

In [32]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data['is_deleted']==1]
conti = predict_data.loc[predict_data['is_deleted']==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1.
 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1.
 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.
 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0.
 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0.

In [33]:
result_test = pd.DataFrame({'y_test': y_test, 'y_pred': y_test_pred})
result_test.head()

,y_test,y_pred
407,1.0,1.0
338,1.0,1.0
260,1.0,1.0
248,1.0,1.0
1161,0.0,0.0


### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

In [35]:
from numpy import result_type
correct = len(result_test.loc[result_test['y_test']==result_test['y_pred']])
data_count = len(result_test)
score_test = correct / data_count
print(score_test)

0.8859315589353612


In [36]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8859315589353612
0.9283903675538656


In [38]:
X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8897338403041825
0.899873257287706


### ノック49：モデルに寄与している変数を確認しよう

In [40]:
importance = pd.DataFrame({'feature_names': X.columns, 'coefficient': model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.338026
1,routine_flg,0.185839
2,period,0.465160
3,campaign_name_入会費無料,0.000000
4,campaign_name_通常,0.009136
5,class_name_デイタイム,0.001711
6,class_name_ナイト,0.000129
7,gender_M,0.000000


### ノック50：顧客の退会を予測しよう

In [42]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = '入会費無料'
class_name = 'オールタイム'
gender = 'M'

In [43]:
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [44]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  "X does not have valid feature names, but"
